# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: Ксенофонтов Григорий Сергеевич

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.
- [Pytorch](https://pytorch.org/) — инструмент для обучения нейросетей.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

#### Тесты

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/"MyDrive"/"Colab Notebooks"/"MMTA FALL 2022"/

data				       __pycache__
download_utils.py		       stackoverflow_similar_questions.zip
GoogleNews-vectors-negative300.bin.gz  task1.ipynb
mmta_tests.py			       test_gt.json


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022')

In [4]:
from mmta_tests import TaskTests
task_tests = TaskTests.from_json(path='/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022/test_gt.json')

### Вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов. Используйте [модель эмбеддингов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которая была обучена с помощью пакета word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.

In [5]:
# from download_utils import download_google_vectors


#download_google_vectors(target_dir='/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022/')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [6]:
import gensim


wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    '/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000,
)

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого проверьте три пункта:
1. Используя метод `.most_similar(positive=..., negative=...)`, найти слово, похожее на `woman`, `king` и непохожее на `man`.
2. Используя метод `.doesnt_match(...)`, найти "белую ворону" в списке `['breakfast, 'dinner', 'lunch', 'cereal']`.
3. Используя метод `.most_similar_to_given(word, [...])`, найти наиболее похожее на `music` слово из списка `['water', 'sound', 'backpack', 'mouse']`.

Прокомментируйте полученные результаты: считаете ли вы их верными и почему.

In [7]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
print(wv_embeddings.most_similar(positive=['king', 'woman'], negative=['man']))
print(wv_embeddings.doesnt_match(['breakfast', 'dinner', 'lunch', 'cereal']))
print(wv_embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse']))

[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('queens', 0.518113374710083), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454), ('royal_palace', 0.5087165832519531)]
cereal
sound


/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Ранжирование вопросов StackOverflow

Давайте посмотрим на данные, которые мы будем использовать в рамках задания. Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (train.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую (валидационную) выборку. Ответьте на следующие вопросы:
1. Сколько пар-дубликатов предоставлено в выборке?
2. Сколько в среднем на каждую пару предоставлено отрицательных примеров?

In [8]:
import tqdm


def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

validation = read_corpus('/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022/data/validation.tsv')

In [9]:
import numpy as np

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

num_samples = len(validation) # Сколько пар-дубликатов предоставлено в выборке?
print(num_samples)

amount_of_negatives_per_sample = np.mean(list(map(lambda x: len(x)-2, validation))) # Сколько в среднем на каждую пару предоставлено отрицательных примеров?
print(amount_of_negatives_per_sample)

3760
998.8106382978723


In [10]:
task_tests.test_validation_corpus(
    num_samples,
    amount_of_negatives_per_sample
)

### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.



In [11]:
import numpy as np


class Embedder:
    
    def __init__(self, embeddings, dim):
        """
            embeddings: word2vec эмбеддинги
            dim: размерность word2vec эмбеддингов. Нужна для задания нулего вектора для пустых вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.embeddings = embeddings
        self.dim = dim
        
    def __call__(self, text, normalize=False):
        """
            Принимает на вход текст и преобразует его в вектор.
            
            text: строка с вопросом
            normalize: при True нужно перед возвращением нормализовать вектор
            
            returns: вектор вопроса
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        cosine = np.zeros(self.dim)
        sentence = text.replace(',', ' ').replace('.', ' ').replace('?', ' ').split(' ')
        sentence = list(filter(lambda el: el != '', sentence))
        n = len(sentence)

        
        for word in sentence:
          try:
            cosine += self.embeddings[word]
          except KeyError:
            n -= 1

        res = cosine / n
        if normalize:
          return  res / np.linalg.norm(res)
        else: 
          return res


In [12]:
embedder = Embedder(wv_embeddings, dim=300)

In [13]:
#task_tests.test_embedder(embedder)

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

В качестве метрики схожести вопросов будем использовать косинусную близость.

В валидационном датасете для каждой пары вопросов-дубликатов у нас есть случайные отрицательные примеры. Для каждого триплета (вопрос, дубликат, отрицательные примеры) будет ранжировать с помощью нашей модели и косинусной близости дубликат и отрицательные примеры и смотреть на позицию дубликата.

#### Hits@K
Довольно простой и легко интерпретируемой метрикой будет количество корректных попаданий дубликата в top "выдачи" для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### Пример оценок

Пусть $N = 1$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

#### Подсчет метрики Hits@k сразу для нескольких k

Чтобы посчитать метрику для нескольких k, не нужно повторно ранжировать нашей моделью вопросы для одного и того же сэмпла. Достаточно посчитать для сэмпла количество **сложных негативов** - отрицательных примеров, оказавшихся в выдаче выше, чем дубликат. Тогда
$$Hits@k = \begin{cases}
    1, & N < k \\
    0, & иначе
   \end{cases},$$
где **N** - количество сложных негативов.

Реализуйте подсчет Hits@k для произвольного набора значений k и заданной валидационной выборки, используя предложенный шаблон.

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
np.seterr(divide='ignore', invalid='ignore')        
    
class Scorer:
    
    def __init__(self, k, embedder):
        """
            k: список значений k, для которых нужно посчитать hits@k
            embedder: объект класса Embedder, умеющий преобразовать текст в вектор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.k = k
        self.embedder = embedder
        
    def _get_hard_negatives(self, q, pos, negs):
        """
            q: текст вопроса
            pos: текст дубликата
            negs: список из текстов случайных вопросов
            
            result: количество сложных отрицательных примеров, оказавшихся выше положительного
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        q_vec = self.embedder(q)
        q_vec = np.nan_to_num(q_vec, nan=0.0).reshape(1, -1)
        pos_vec = self.embedder(pos)
        pos_vec = np.nan_to_num(pos_vec, nan=0.0).reshape(1, -1)
        negs_vecs = np.array(list(map(self.embedder, negs)))
        negs_vecs = np.nan_to_num(negs_vecs, nan=0.0)

        pos_similarity = cosine_similarity(pos_vec, q_vec).item()
        negs_similarity = cosine_similarity(negs_vecs, q_vec).reshape(-1)
        return np.sum(negs_similarity > pos_similarity)


    
    def __call__(self, samples, verbose=False):
        """
            samples: список из списков вида [q, pos, neg1, neg2, ...]. Наша валидационная выборка
            verbose: выводить progressbar подсчета метрики с помощью tqdm
            
            result: словарь вида {k: hits@k}
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        hits = {k: 0 for k in self.k}
        range_ = tqdm.tqdm(samples) if verbose else samples
        for sample in range_:
          N = self._get_hard_negatives(sample[0], sample[1], sample[2:])
          hits = {hit[0]: hit[1] + 1 if N < hit[0] else hit[1] for hit in hits.items()}
        n = len(samples)
        return {hit[0]: hit[1]/n for hit in hits.items()}

In [15]:
scorer = Scorer([1, 5, 10], embedder=embedder)
scorer(validation[:10], verbose=True)

100%|██████████| 10/10 [00:00<00:00, 15.22it/s]


{1: 0.2, 5: 0.3, 10: 0.5}

In [16]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation, verbose=True)

100%|██████████| 3760/3760 [04:05<00:00, 15.30it/s]


In [17]:
print(hits)

{1: 0.23297872340425532, 5: 0.3398936170212766, 10: 0.3941489361702128, 100: 0.598404255319149, 500: 0.8295212765957447, 1000: 1.0}


In [18]:
#task_tests.test_scorer(hits)

### Предобработка текста

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации и всевозможные плохие символы на пробелы;
- Удалить стопслова.
- Удалить слова с длиной меньше трех букв

Реализуйте предобработку текста, используя предложенный шаблон.

In [19]:
import re
    
    
class TextPreprocessor:
    
    def __init__(self, characters, min_word_length=0, stopwords=None):
        """
            characters: список плохих символов
            min_word_length: минимальная допустимая длина для слов
            stopwords: множество фоновых слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.characters = set(characters)
        self._bad_reg_exp = re.compile('[' + ''.join("\\" + c for c in characters) + ']')
        self.min_word_length = min_word_length
        self.stopwords = stopwords

    def __call__(self, text):
        """
            text: текст для обработки
            
            returns: обработанный текст
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        text = text.lower()
        # print('[{}]'.format('\\'.join(self.characters)))
        text = self._bad_reg_exp.sub(' ', text)

        sentence = text.split()
        sentence = list(filter(lambda word: word not in self.stopwords and len(word) >= self.min_word_length, sentence))
        return ' '.join(sentence)

In [20]:
task_tests.test_text_preprocessor(TextPreprocessor)

Множество фоновых слов можно взять из **nltk** с помощью `nltk.corpus.stopwords.words`, выкидываемые плохие символы и пунктуацию следует подобрать самостоятельно.

Обработайте текст и продемонстрируйте улучшение качества:

In [21]:
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
    

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
sym_to_remove = ('?', '!', ',', '.' , ';', ':', '[', ']', '(', ')', '{', '}', 
                "''", '>', '<', '%', '^', '$', '*', '#', '@', '~', '/', '\\',
               '+', '-', '=')
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)
preprocessor = TextPreprocessor(sym_to_remove, min_word_length=3, stopwords=set(stopwords.words('english')))
preprocessed = []
for sample in validation:
  preprocessed.append(list(map(preprocessor, sample)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
hits = scorer(preprocessed, verbose=True)
hits

100%|██████████| 3760/3760 [03:04<00:00, 20.42it/s]


{1: 0.3281914893617021,
 5: 0.4678191489361702,
 10: 0.5183510638297872,
 100: 0.6914893617021277,
 500: 0.8534574468085107,
 1000: 1.0}

Одним из критериев получения полных баллов является значение **hits@500** $\geqslant 0.82$ до предобработки текста и $\geqslant 0.85$ после предобработки.

## Часть 2. Представления для неизвестных слов. (4 балла)

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм #wh, whe, her, ere, re#

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

### Создание триграммного токенизатора

Для начала, нам нужно:
1. Пройтись по известным в word2vec словам и составить множество триграмм, для которых будем обучать векторы
2. Составить маппинг из триграмм в индексы
3. Реализовать преобразование произвольного слова в список триграмм
4. Реализовать преобразование произвольного слова в список индексов триграмм

Для реализации всех этих пунктов предлагается использовать шаблон, приведенный ниже.

In [23]:
class TrigramTokenizer:
    
    def __init__(self, words):
        """
            Формируем множество всевозможных триграмм, встречающихся в словах из words.
            Делаем маппинг триграмм в индексы.
            
            words: список слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.trigramms = []
        for word in tqdm.tqdm(words):
          for trigramm in TrigramTokenizer._get_trigrams(word):
            if trigramm not in self.trigramms:
              self.trigramms.append(trigramm)

        
    @property
    def vocab_size(self):
        """
            returns: колчиество триграмм, для которых мы завели индекс.
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.trigramms)
    
    @staticmethod
    def _get_trigrams(word):
        """
            word: слово
            
            returns: список триграмм для word
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        if len(word) < 2:
            return []

        trigramms = []
        for i in range(len(word)):
          if i == 0:
            trigram = "".join(['#', word[i], word[i + 1]])
          elif i == len(word) - 1:
            trigram = "".join([word[i - 1], word[i], '#'])
          else:
            trigram = "".join([word[i - 1], word[i], word[i + 1]])
          trigramms.append(trigram)
        return trigramms
        
    def __call__(self, word):
        """
            word: слово
            
            returns: список индексов триграмм для слова word, которые нашлись в маппинге
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        filtered = filter(lambda x: x in self.trigramms, TrigramTokenizer._get_trigrams(word))
        return list(map(lambda x: self.trigramms.index(x), filtered))
              

In [24]:
task_tests.test_trigram_tokenizer(TrigramTokenizer)

Для создания токенизатора используйте обработанный с помощью TextProcessor текст. 

**Важно:** в токенизатор нужно подавать только слова, известные word2vec'у.

In [25]:
# wv_embeddings_dict = dict(zip(map(preprocessor, wv_embeddings.index2entity), wv_embeddings.vectors))
wv_embeddings_dict = dict(zip(wv_embeddings.vocab.keys(), wv_embeddings.vectors))

In [26]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
# w2v_vocab = map(preprocessor, wv_embeddings.vocab.keys())
w2v_vocab = list(wv_embeddings.vocab.keys())
tri_tokenizer = TrigramTokenizer(w2v_vocab)

100%|██████████| 500000/500000 [05:16<00:00, 1578.51it/s]


### Создание датасета с w2v векторами и списками индексов триграмм

Мы будем обучать триграммную модель в нейросетевом фреймворке pytorch. Для этого нам нужно создать свой датасет.

Он должен:
1. Принимать список слов, word2vec и уже созданный триграммный токенизатор.
2. Выдавать пары вида (эмбеддинг для слова из word2vec, список индексов триграмм для этого слова)

Реализовать датасет нужно в шаблоне, приведенном ниже.

In [27]:
from torch.utils.data import Dataset


class TrainTrigramDataset(Dataset):
    
    def __init__(self, vocab, w2v_embeddings, tri_tokenizer):
        """
            Формируем выборку для обучения триграммной модели.
            ЗАРАНЕЕ считаем маппинг в список индексов для всех известных в word2vec слов.
            
            vocab: список слов
            w2v_embeddings: no comments
            tri_tokenizer: токенизатор триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab
        self.w2v_embeddings = w2v_embeddings
        self.tri_tokenizer = tri_tokenizer 
                
    def __len__(self):
        """
            returns: возвращает количество слов, вошедших в маппинг (размер словаря)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.tri_tokenizer.vocab_size
    
    def __getitem__(self, idx):
        """
            returns: w2v эмбеддинг для idx-го слова в датасете, список соответствующих ему триграмм (тензоры)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.w2v_embeddings[self.vocab[idx]], self.tri_tokenizer(self.vocab[idx])

    
ds = TrainTrigramDataset(list(w2v_vocab), wv_embeddings_dict, tri_tokenizer)

In [28]:
task_tests.test_dataset(ds, list(w2v_vocab), wv_embeddings_dict, tri_tokenizer)

### Создание DataLoader'а и Collator'а

Нас интересуют в первую очередь четыре параметра при создании DataLoader:
1. Датасет. Реализует интерфейс массива - можно узнать длину и получить элемент с индексом, меньшим длины.
2. batch_size. Задает размера батча (количества сэмплов, идущих одновременно в модель).
3. shuffle. При shuffle == True каждую эпоху при итерировании по даталоадеру мы будем получать сэмплы в произвольном порядке.
4. collate_fn. Этот параметр позволяет задать кастомную логику "склеивания" сэмплов из датасета в батч.

В качестве модели мы будем использовать слой **torch.nn.EmbeddingBag**. Он принимает на вход список индексов и список сдвигов, начинающийся с нуля.

Нужно наш список списков индексов триграмм превратить в соответствующий формат, преобразовать векторы слов и два списка (индексов и сдвигов) в pytorch тензоры (torch.tensor).

Реализуйте следующую функцию:

In [29]:
import torch
from torch.utils.data import DataLoader


def collate_fn(batch):
    """
        batch: список из элементов датасета, e.g. [ds[i] for i in [2, 3, 1, 15]]
        
        returns: w2v эмбеддинги, индексы триграмм, сдвиги для триграмм
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    w2vs = []
    tri_idxs = []
    offests = [0]
    for item in batch:
      w2vs.append(item[0])
      tri_idxs.extend(item[1])
      offests.append(offests[-1] + len(item[1]))
    offests.pop()
    return torch.tensor(w2vs), torch.tensor(tri_idxs), torch.tensor(offests)

In [30]:
task_tests.test_dataloader(ds, collate_fn, embedding_dim=300)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


### Создание модели

При создании модели мы обычно наследуемся от **torch.nn.Module** и создаем нужные нам слои как атрибуты объекта нашего класса.

В данном случае предлагается для формирования эмбеддингов использовать **torch.nn.EmbeddingBag**.

Реализуйте предложенный шаблон:

In [31]:
from torch import nn


class TrigramModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество триграмм, для которых обучаются эмбеддинги
            embedding_dim: размерность эмбеддингов триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self._num_embeddings = num_embeddings
        self._embedding_dim = embedding_dim
        super().__init__()
        self.net = nn.EmbeddingBag(num_embeddings, embedding_dim)
        
    @property
    def embedding_dim(self):
        """
            returns: размерность эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self._embedding_dim
    
    @property
    def num_embeddings(self):
        """
            returns: количество эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self._num_embeddings
    
    def forward(self, trigrams, offsets):
        """
            trigrams: список индексов триграмм (тензор)
            offsets: список сдвигов (тензор)
            
            returns: эмбеддинги слов, составленные из триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        out = self.net(trigrams, offsets)
        return out.double()

    
model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size)


In [32]:
task_tests.test_trigram_model(model)

### Создание пайплайна обучения

Далее необходимо совместить все наработки в единый пайплайн обучения, добавив также критерий для оптимизации и оптимизатор. 

Предлагается:

1. В качестве оптимизатора использовать Adam (можно попробовать подобрать learning rate / weight decay)
2. В качестве критерия оптимизации взять nn.MSELoss (можно также закодить лосс самому)
3. Для даталоадера выбрать небольшой батч сайз (32, 64, 128, 256)
4. Десяти эпох должно быть достаточно для хорошего качества

Реализуйте предложенный шаблон.

In [33]:
import time
from torch import optim

    
class Trainer:
    
    def __init__(self, model, criterion, optimizer):
        """
            model: триграммная модель
            criterion: функционал ошибки, принимает на вход w2v эмбеддинги и триграммные эмбеддинги
            optimizer: оптимизатор для модели
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        
    def _train_step(self, dataloader):
        """
            Делаем один проход по даталоадеру, с бэкпропом
            
            dataloader: даталоадер с тренировочными данными
            
            returns: лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        for emb, idx, offset in tqdm.tqdm(dataloader):
          emb = emb.to(self.device)
          idx = idx.to(self.device)
          offset = offset.to(self.device)
          self.optimizer.zero_grad()
          out = self.model(idx, offset)
          loss = self.criterion(out, emb.double())
          loss.backward()
          self.optimizer.step()
        return loss.item()

    def train(self, dataloader, n_epochs, verbose=False):
        """
            dataloader: тренировочный даталоадер
            n_epochs: количество эпох
            verbose: выводить лосс каждую эпоху или нет
            
            returns: список лоссов
        """
        start = time.time()
        losses = []
        for epoch in range(n_epochs):
            loss = self._train_step(dataloader)
            losses.append(loss)
            if verbose:
                print(f'epoch: {epoch + 1:>2}, loss: {loss:.4f}, time: {time.time() - start:.4f}')
        return losses


###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size).to(device)
trainer = Trainer(model, nn.MSELoss(), optim.Adam(model.parameters(), lr=0.05))
dataloader = DataLoader(ds, 128, True, collate_fn=collate_fn, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [34]:
 trainer.train(dataloader, 10, verbose=True)

100%|██████████| 428/428 [00:29<00:00, 14.34it/s]


epoch:  1, loss: 0.0822, time: 29.8590


100%|██████████| 428/428 [00:31<00:00, 13.52it/s]


epoch:  2, loss: 0.0518, time: 61.5331


100%|██████████| 428/428 [00:29<00:00, 14.33it/s]


...

100%|██████████| 428/428 [00:29<00:00, 14.32it/s]


epoch:  9, loss: 0.0442, time: 272.9835


100%|██████████| 428/428 [00:29<00:00, 14.30it/s]

epoch: 10, loss: 0.0451, time: 302.9195


[0.08217375979746766,
 0.05184470402124019,
 0.04366217737976522,
 0.037773075982692444,
 0.03779639360080277,
 0.04392957690525796,
 0.042851675579719535,
 0.05285354578981311,
 0.044162892548117394,
 0.04514925571556161]

### Получение векторов неизвестных слов. Инференс модели

Теперь, когда мы обучили модель, нам необходимо применить её для всех неизвестных слов, т.е. получить для них эмбеддинги.

Т.к. для этих слов у нас нет word2vec эмбеддингов, то dataset и collator для обучения не подходят для инференса. Необходимо реализовать датасет и коллатор для инференса по следующим шаблонам:

In [35]:
class InferenceTrigramDataset:
    
    def __init__(self, vocab, tri_tokenizer):
        """
            Датасет с неизвестными словами
            
            vocab: список слов
            tri_tokenizer: триграммный токенизатор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab
        self.tri_tokenizer = tri_tokenizer
        
    def __len__(self):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.vocab)
    
    def __getitem__(self, idx):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.vocab[idx], self.tri_tokenizer(self.vocab[idx])
    
    
def inference_collate_fn(trigrams):
    """
        trigrams: список списков индексов триграмм
        
        returns: список индексов, список сдвигов триграмм \\\ Я добавил слова для удобства
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    idxs = []
    offests = [0]
    words = []
    for word, idx in trigrams:
      words.append(word)
      idxs.extend(idx)
      offests.append(offests[-1] + len(idx)) 
    offests.pop()
    return words, torch.tensor(idxs), torch.tensor(offests)

Теперь у нас есть всё необходимое, чтобы осуществить инференс. Не забудь перед инференсом перевести модель в режим эвала (**model.eval**), а также использовать контекстный менеджер **torch.no_grad**.

После инференса сформируйте словарь из известных в word2vec слов и их эмбеддингов, затем дополните его эмбеддингами для неизвестных слов, полученными после инференса.

In [36]:
unknown = set()
w2v_vocab_set = set(w2v_vocab)
for sample in tqdm.tqdm(preprocessed):
  # sample = map(preprocessor, sample)
  for sentence in sample:
    words = set(filter(lambda word: word not in w2v_vocab_set, sentence.split()))
    unknown.update(words)
unknown = list(unknown)

100%|██████████| 3760/3760 [00:06<00:00, 544.95it/s]


In [37]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
model.cpu().eval()
new_ebeddings = {}
infDS = InferenceTrigramDataset(unknown, tri_tokenizer)
dl = DataLoader(infDS, shuffle=True, collate_fn=inference_collate_fn)
with torch.no_grad():
  for word, idx, offset in dl:
    if idx.nelement() != 0:
      embedding = model(idx, offset)
      new_ebeddings[word[0]] = embedding.numpy().squeeze(0)

In [38]:
print(unknown[:10])

['typenamehandling', '"bold"', 'idbtransaction', 'backgroundresource', 'subviews', 'cassandra', 'sl4a', '`__gnu_cxx', 'patchvalue', '`user_registration_path']


In [39]:
  w2v_trigram = {**new_ebeddings, **wv_embeddings_dict}

Используя **Scorer** и **Embedder**, получите новые значения метрик для валидации:

In [40]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
embedder = Embedder(w2v_trigram, dim=300)
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(preprocessed, verbose=True)

100%|██████████| 3760/3760 [02:07<00:00, 29.44it/s]


In [41]:
hits

{1: 0.4023936170212766,
 5: 0.550531914893617,
 10: 0.6,
 100: 0.7587765957446808,
 500: 0.9050531914893617,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.89$.

## Часть 3. Обучение векторных представлений для целевой задачи. (4 баллов)

Предполагается, что в этой части используются TextPreprocessor, Embedder, Scorer из предыдущих частей.

Для обучения на целевую задачу нам понадобится обучающая выборка. Считайте её с диска, предобработайте текст вопросов

In [42]:
train = []
for questions in tqdm.tqdm(read_corpus('/content/drive/MyDrive/Colab Notebooks/MMTA FALL 2022/data/train.tsv')):
    train.append([preprocessor(text) for text in questions])

100%|██████████| 1000000/1000000 [00:11<00:00, 85852.03it/s]


Необходимо создать **токенизатор для текста** - составить словарь и сделать маппинг из слов в индексы.

In [43]:
class TextTokenizer:
    
    def __init__(self, vocab):
        """
            vocab: множество слов, встретившихся в обучающей выборке
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab

    @property
    def vocab_size(self):
        """
            returns: количество слов в словаре
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.vocab)
    
    def __call__(self, text):
        """
            text: текст
            
            returns: список индексов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        filtered = filter(lambda x: x in self.vocab, text.split())
        return list(map(self.vocab.index, filtered))

Составление словаря и токенизатора

In [44]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
# vocab = set()
# for sample in tqdm.tqdm(train):
#   for text in sample:
#     vocab.update(text.split(' '))

# vocab = list(vocab)
vocab = dict()
for sample in tqdm.tqdm(train):
  for text in sample:
    for word in text.split(' '):
      if word not in vocab:
        vocab[word] = 1
      else:
        vocab[word] +=  1
print(f'There are {len(vocab)} words')
sorted_values = list(sorted(vocab.values()))
left_thr = sorted_values[int(0.5 * len(vocab))]
right_thr = sorted_values[min(int(1 * len(vocab)), len(vocab) - 1)]
fil_words = list(filter(lambda word: vocab[word] <= right_thr and vocab[word] >= left_thr, vocab))
print(f'There are {len(fil_words)} filtered words')
                
vocab = fil_words

textTokenizer = TextTokenizer(vocab)

100%|██████████| 1000000/1000000 [00:04<00:00, 213212.06it/s]

There are 162379 words
There are 81322 filtered words


Нам также понадобится **новый датасет для обучения**. Для применения метода NT-Exent нам не нужно "майнить негативы", поэтому датасет надо сформировать как массив из пар-дубликатов.

Так как данные в обучающей выборке содержат множества дубликатов (т.е. все дубликаты сгруппированы в списки), есть несколько способов сформировать итоговый датасет:
1. Оставить из каждого множества дубликатов какие-нибудь случайные два (или просто первые два вопроса)
2. Для первого вопроса в множестве взять все остальные как дубликаты (N вопросов-дубликатов - N-1 пара). Тогда мы увидим каждый вопрос хотя бы один раз при обучении
3. Составить всевозможные уникальные пары-дубликаты из этих множеств (т.е. первый вопрос и все остальные вопросы, второй вопрос и все остальные, кроме первого).

Каждый следующий способ, начиная с первого, раздувает выборку по размеру, но возможно дает прирост к качеству решения задачи.

Реализуйте выбранный вами подход, используя предолженный шаблон:

In [45]:
class QuestionDuplicatesDataset(Dataset):
    
    def __init__(self, question_pairs, tokenizer):
        """
            question_pairs: список из пар вопросов-дубликатов
            tokenizer: объект класса TextTokenizer
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.question_pairs = question_pairs
        self.tokenizer = tokenizer
            
    def __len__(self):
        """
            returns: количество пар-дубликатов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.question_pairs)
    
    def __getitem__(self, idx):
        """
            returns: (вопрос, дубликат), idx-ю пару в датасете
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return list(map(self.tokenizer, self.question_pairs[idx]))

Также нужно подготовить **даталоадер** (а именно - коллатор для даталоадера) по аналогии со второй частью задания.

In [46]:
def get_ids_and_offsets(questions):
    """
        questions: список из токенизированных вопросов
        
        returns: (ids, offsets), где ids - вытянутый список индексов слов в вопросах из батча, offsets - сдвиги
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    ids = []
    offsets = [0]
    for question in questions:
      ids.extend(question)
      offsets.append(offsets[-1] + len(question))
    offsets.pop()
    return torch.tensor(ids), torch.tensor(offsets)



def collate_fn(batch):
    """
        batch: список из пар токенизированных вопросов-дубликатов [(question, duplicate), ...]
        
        returns: (question_ids, question_offsets), (duplicate_ids, duplicate_offsets)
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    questions, duplicates = zip(*batch)
    return get_ids_and_offsets(questions), get_ids_and_offsets(duplicates)


Поделите выборку на трейн и валидацию, используя train_test_split, затем **создайте датасеты и даталоадеры** для обучения и валидации. Сколько пар-дубликатов получилось в датасете для обучения?

In [47]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
from sklearn.model_selection import train_test_split

ds = QuestionDuplicatesDataset(train, textTokenizer)
ds_train, ds_test, _, _ = train_test_split(ds, [1] * len(ds), shuffle=False, random_state=42)



In [48]:
dl_train = DataLoader(ds_train, batch_size=2048, shuffle=True, collate_fn=collate_fn, num_workers=4)
dl_test = DataLoader(ds_test, batch_size=2048, shuffle=False, collate_fn=collate_fn, num_workers=4)

print(f'Train dataset contains:\t {len(ds_train)} samples')
print(f'Test dataset contains:\t {len(ds_test)} samples')

Train dataset contains:	 750000 samples
Test dataset contains:	 250000 samples


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


С помощью предложенного шаблона **задайте модель** для преобразования вопросов в векторы.

In [57]:
from torch import nn


class DssmLikeModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество слов, для которых обучаем эмбеддинги
            embedding_dim: размерность эмбеддинга
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.net = nn.EmbeddingBag(num_embeddings, embedding_dim)
        
    def forward(self, ids, offsets):
        """
            ids: вытянутая посл-ть индексов слов вопросов, попавших в батч
            offsets: сдвиги для вопросов, попавших в батч
            
            returns: векторы вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.net(ids, offsets)

Создание модели

In [68]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
model = DssmLikeModel(len(vocab), embedding_dim=wv_embeddings.vector_size)

**Критерий оптимизации** для NTExentLoss выглядит как:

$$\mathcal{L}(Q, D) = -0.5 \log diag(softmax(QD^T / \alpha)) - 0.5 \log diag(softmax(DQ^T / \alpha)),$$

где:
* $Q \in \mathbb{R}^{b \times d}$ - эмбеддинги вопросов, 
* $D \in \mathbb{R}^{b \times d}$ - эмбеддинги соответствующих вопросам дубликатов,
* $b$ - количество пар (вопрос, дубликат), $d$ - размерность эмбеддингов, $\alpha$ - гиперпараметр лосса. 
* Softmax берется по рядам
* Матрицы $Q, D$ содержат нормированные эмбеддинги, т.е. считается именно косинус.

In [69]:
class NTExentLoss(nn.Module):
    
    def __init__(self, alpha=1., eps=1e-8):
        """
            alpha: коэффициент, на который мы делим скоры перед софтмаксом
            eps: ||v|| = min(eps, ||v||)
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.alpha = alpha
        self.eps = eps
        
    def _normalize(self, embeddings):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            
            returns: матрица такого же размера, но с нормироваными векторами
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return embeddings / (torch.norm(embeddings, dim=1).reshape(-1, 1) + self.eps)
    
    def forward(self, embeddings, positives):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            positives: матрица такого же размера, с позитивами для векторов из матрицы embeddings
            
            returns: NT-Exent loss
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        embeddings = self._normalize(embeddings)
        positives = self._normalize(positives)
        return torch.mean(-0.5 * torch.log(torch.diag(torch.softmax(embeddings @ positives.t() / self.alpha, dim=1), 0)) - \
                          -0.5 * torch.log(torch.diag(torch.softmax(positives @ embeddings.t() / self.alpha, dim=1), 0)))
        

**Создайте пайплайн** для обучения и валидации, используя предложенный шаблон.

Залогируйте с помощью **torch.utils.tensorboard.SummaryWriter** две величины:
1. Лосс для каждого батча
2. Лосс на валидации для каждой эпохи

In [70]:
import time
import shutil
import os
import torch

from torch.utils.tensorboard import SummaryWriter
    
    
class Trainer:
    
    def __init__(
            self, 
            model, 
            optimizer, 
            criterion, 
            logdir=None, 
            device=None
    ):
        """
            model: объект класса DssmModel
            optimizer: оптимизатор
            criterion: критерий оптимизации
            logdir: директория, в которую SummaryWriter должен писать логи
            device: девайс (cpu или cuda), на котором надо производить вычисления
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.device = device
        if self.device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.criterion = self.criterion.to(self.device)
        self.logdir = logdir
        self._writer = SummaryWriter()
    
    def _calculate_loss(self, batch):
        """
            batch: батч из индексов и сдвигов для вопросов и их дубликатов
            
            returns: посчитанный для батча лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        (question_idx, question_offs), (duplicate_idx, duplicate_offs) = batch
        return self.criterion(self.model(question_idx, question_offs),
                              self.model(duplicate_idx, duplicate_offs))
        
    def _train_step(self, dataloader):
        """
            dataloader: даталоадер для обучения
            
            returns: лосс на датасете для обучения
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        for (question_idx, question_offs), (duplicate_idx, duplicate_offs) in tqdm.tqdm(dataloader):
            question_idx = question_idx.to(self.device)
            question_offs = question_offs.to(self.device)
            duplicate_idx = duplicate_idx.to(self.device)
            duplicate_offs = duplicate_offs.to(self.device)
            loss = self._calculate_loss(((question_idx, question_offs), (duplicate_idx, duplicate_offs)))
            self._writer.add_scalar("Loss/train", loss.detach().item())
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        return loss.detach().item()

    def _eval_step(self, dataloader):
        """
            dataloader: даталоадер для валидации
            
            returns: лосс на валидации
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        avg_loss = 0
        with torch.no_grad():
            for i, ((question_idx, question_offs), (duplicate_idx, duplicate_offs)) in enumerate(dataloader):
                question_idx = question_idx.to(self.device)
                question_offs = question_offs.to(self.device)
                duplicate_idx = duplicate_idx.to(self.device)
                duplicate_offs = duplicate_offs.to(self.device)
                avg_loss = (avg_loss * i + self._calculate_loss(((question_idx, question_offs), (duplicate_idx, duplicate_offs))).item()) / (i + 1)
        return avg_loss
    
    def train(self, dataloaders, n_epochs, verbose=False):
        """
            dataloaders: словарь вида {'train': train_dataloader, 'eval': eval_dataloader}
            n_epochs: количество эпох обучения
            verbose: нужно ли выводить каждую эпоху информацию про лоссы
        """
        start = time.time()
        for epoch in range(n_epochs):
            train_loss = self._train_step(dataloaders['train'])
            
            eval_loss = self._eval_step(dataloaders['eval'])
            if self._writer is not None:
                self._writer.add_scalar('eval/loss', eval_loss)
                
            if verbose:
                print(
                    'epoch: {:>2}, train loss: {:.4f}, eval loss: {:.4f}, time: {:.4f}' \
                        .format(epoch + 1, train_loss, eval_loss, time.time() - start)
                )
                    


In [71]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

optimizer = optim.Adam(model.parameters(), lr=3e-2, weight_decay=0.0)
trainer = Trainer(model, optimizer, NTExentLoss(), 'training_log', None)

Предлагается использовать для оптимизации Адам и обучать модель 10-60 эпох.

Для этой части задания GPU даёт существенное ускорение при обучении, поэтому стоит по возможности делать обучение с большим batch size'ом и на GPU.

In [72]:
 ###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
trainer.train({'train': dl_train, 'eval': dl_test}, 60, verbose=True)

100%|██████████| 367/367 [00:08<00:00, 44.94it/s]


epoch:  1, train loss: 0.0020, eval loss: -0.0042, time: 9.5338


100%|██████████| 367/367 [00:08<00:00, 44.73it/s]


epoch:  2, train loss: -0.0042, eval loss: -0.0059, time: 19.1362


100%|██████████| 367/367 [00:08<00:00, 44.95it/s]


...

epoch: 58, train loss: -0.0348, eval loss: -0.0094, time: 555.0637


100%|██████████| 367/367 [00:08<00:00, 45.52it/s]


epoch: 59, train loss: -0.0529, eval loss: -0.0096, time: 564.4947


100%|██████████| 367/367 [00:08<00:00, 45.45it/s]


epoch: 60, train loss: -0.0317, eval loss: -0.0096, time: 573.9479


Осталось достать из модели обученные под задачу векторы слов, составить маппинг слов в векторы, создать **Embedder** и **Scorer** и провалидировать качество на нашей исходной валидации, которой мы пользовались в первых двух частях.

Чтобы достать из модели веса, можно использовать `model._embeddings.weight.cpu().detach().numpy()`

In [73]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
import warnings
warnings.filterwarnings("ignore")

embeddings = model.net.weight.cpu().detach().numpy()
embedder = Embedder({word : emb for word, emb in zip(list(vocab), embeddings)}, dim=wv_embeddings.vector_size)
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(preprocessed, verbose=True)

100%|██████████| 3760/3760 [02:15<00:00, 27.84it/s]


In [74]:
hits

{1: 0.15824468085106383,
 5: 0.2125,
 10: 0.23031914893617023,
 100: 0.34867021276595744,
 500: 0.6590425531914894,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.98$.

## Дополнительная часть (до 3 баллов)

Каждый из пунктов при успешном выполнении гарантирует как минимум один дополнительный балл. Максимум вам будет зачтено три пункта. 
1. Обучить триграммную модель на косинусную близость вместо евклидового расстояния, получить прирост качества (относительно триграммной модели с MSE)
2. Обучить в качестве триграммной модели char-biLSTM вместо мешка векторов триграмм, получить прирост качества (относительно триграммной модели с таким же критерием оптимизации)
3. Усложнить модель в части 3, добавить к мешку вектора словесных униграмм также мешок векторов словесных биграмм и мешок векторов буквенных триграмм (сделать модель более похожей на настоящий dssm), получить прирост качества
4. Модифицировать модель в части 3 произвольным образом (добавить MLP, нормализации, дропаут, сделать bilstm поверх последовательности векторов слов, трансформер и т.д.), получить прирост качества
5. Сделать модель с ранним связыванием (early fusion) - векторы вопросов конкатенируются и проходят через MLP (с возможными модификациями) перед созданием предсказания. Hint: возможно стоит предобучить эмбеддинги слов с помощью NT-Exent перед обучением финальной модели.